In [16]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
data = pd.read_csv('datasets/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')

In [3]:
data[" Label"].value_counts()

 Label
DDoS      128027
BENIGN     97718
Name: count, dtype: int64

In [4]:
data = data.fillna(0)
nan_counts = data.isna().sum()
nan_columns = nan_counts[nan_counts > 0]
print(nan_columns)

Series([], dtype: int64)


In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data[' Label'] = le.fit_transform(data[' Label'])

In [6]:
data = data[~np.isinf(data).any(axis=1)]  # Drop rows with infinite values

In [7]:
data

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225740,61374,61,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
225741,61378,72,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
225742,61375,75,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
225743,61323,48,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0


In [8]:
features = data.drop(columns=[" Label"])
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)
scaled_data = pd.DataFrame(scaled_features, columns=features.columns)
scaled_data[" Label"] = data[" Label"].values
print(scaled_data.head())

    Destination Port   Flow Duration   Total Fwd Packets  \
0           0.837225    3.333335e-08            0.000518   
1           0.840109    9.166671e-07            0.000000   
2           0.840124    4.416669e-07            0.000000   
3           0.705548    2.916668e-07            0.000000   
4           0.837194    3.333335e-08            0.000518   

    Total Backward Packets  Total Length of Fwd Packets  \
0                  0.00000                     0.000066   
1                  0.00034                     0.000033   
2                  0.00034                     0.000033   
3                  0.00034                     0.000033   
4                  0.00000                     0.000066   

    Total Length of Bwd Packets   Fwd Packet Length Max  \
0                      0.000000                0.000514   
1                      0.000001                0.000514   
2                      0.000001                0.000514   
3                      0.000001                0

Feature extraction

In [9]:
# Calculate correlation with the label
correlation = scaled_data.corr()[' Label'].abs()  # Absolute correlation with the label
correlation_threshold = 0.3  # Example threshold for correlation

# Select features based on correlation threshold
selected_features_correlation = correlation[correlation > correlation_threshold].index.tolist()
selected_features_correlation.remove(' Label')  # Remove the target variable

X = scaled_data.drop(columns=' Label')
y = scaled_data[' Label']

# Fit a Random Forest model to get feature importances
rf_model = RandomForestClassifier()
rf_model.fit(X, y)

# Get feature importances
feature_importances = rf_model.feature_importances_

# Create a DataFrame for feature importances
feature_importance_df = pd.DataFrame({'Feature': data.drop(' Label', axis=1).columns,
                                       'Importance': feature_importances})

# Set a threshold for feature importance
importance_threshold = 0.01  # Example threshold for feature importance

# Select features based on importance threshold
selected_features_importance = feature_importance_df[feature_importance_df['Importance'] > importance_threshold]['Feature'].tolist()

# Combine selected features from both methods
selected_features = list(set(selected_features_correlation) | set(selected_features_importance))


In [14]:
XX = scaled_data[selected_features]
YY =scaled_data[' Label']

In [11]:
XX

,Bwd Packet Length Std,Destination Port,Init_Win_bytes_backward,Subflow Bwd Packets,Bwd Packet Length Min,Max Packet Length,Avg Fwd Segment Size,Init_Win_bytes_forward,Fwd Packet Length Std,Subflow Fwd Packets,...,Down/Up Ratio,Fwd Header Length,Total Fwd Packets,Subflow Fwd Bytes,Bwd IAT Total,Packet Length Variance,Bwd Packet Length Mean,URG Flag Count,Fwd Packet Length Mean,Flow IAT Std
0,0.0,0.837225,0.000000,0.00000,0.00000,0.000514,0.001552,0.000519,0.0,0.000518,...,0.000000,0.001015,0.000518,0.000066,0.0,0.0,0.000000,0.0,0.001552,0.0
1,0.0,0.840109,0.003922,0.00034,0.00411,0.000514,0.001552,0.000458,0.0,0.000000,...,0.142857,0.000508,0.000000,0.000033,0.0,0.0,0.001034,1.0,0.001552,0.0
2,0.0,0.840124,0.003922,0.00034,0.00411,0.000514,0.001552,0.000458,0.0,0.000000,...,0.142857,0.000508,0.000000,0.000033,0.0,0.0,0.001034,1.0,0.001552,0.0
3,0.0,0.705548,0.005035,0.00034,0.00411,0.000514,0.001552,0.000488,0.0,0.000000,...,0.142857,0.000508,0.000000,0.000033,0.0,0.0,0.001034,1.0,0.001552,0.0
4,0.0,0.837194,0.000000,0.00000,0.00000,0.000514,0.001552,0.000504,0.0,0.000518,...,0.000000,0.001015,0.000518,0.000066,0.0,0.0,0.000000,0.0,0.001552,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225706,0.0,0.936550,0.003876,0.00034,0.00411,0.000514,0.001552,0.004410,0.0,0.000000,...,0.142857,0.000508,0.000000,0.000033,0.0,0.0,0.001034,1.0,0.001552,0.0
225707,0.0,0.936611,0.003876,0.00034,0.00411,0.000514,0.001552,0.004410,0.0,0.000000,...,0.142857,0.000508,0.000000,0.000033,0.0,0.0,0.001034,1.0,0.001552,0.0
225708,0.0,0.936565,0.003876,0.00034,0.00411,0.000514,0.001552,0.004410,0.0,0.000000,...,0.142857,0.000508,0.000000,0.000033,0.0,0.0,0.001034,1.0,0.001552,0.0
225709,0.0,0.935772,0.000000,0.00000,0.00000,0.000514,0.001552,0.072021,0.0,0.000518,...,0.000000,0.001015,0.000518,0.000066,0.0,0.0,0.000000,0.0,0.001552,0.0


In [15]:
YY

0         0
1         0
2         0
3         0
4         0
         ..
225706    0
225707    0
225708    0
225709    0
225710    0
Name:  Label, Length: 225711, dtype: int64

In [17]:
models = {
    'Random Forest': RandomForestClassifier(),
    'Logistic Regression': LogisticRegression(max_iter=200),
    'Support Vector Classifier': SVC(),
    'Decision Tree': DecisionTreeClassifier()
}

In [18]:
results = {}

for model_name, model in models.items():
    # Perform cross-validation
    cv_scores = cross_val_score(model, XX, YY , cv=5)  # 5-fold cross-validation
    results[model_name] = {
        'Mean Accuracy': cv_scores.mean(),
        'Std Deviation': cv_scores.std()
    }

In [19]:
results_df = pd.DataFrame(results).T
print(results_df)

                           Mean Accuracy  Std Deviation
Random Forest                   0.999672       0.000330
Logistic Regression             0.982925       0.010284
Support Vector Classifier       0.985747       0.008828
Decision Tree                   0.999646       0.000256


In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


# Train-test split
X_train, X_test, y_train, y_test = train_test_split(XX, YY, test_size=0.2, random_state=42)

# Create Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Cross-validation
cv_scores = cross_val_score(rf_model, X, y, cv=5)  # 5-fold cross-validation
print(f"Cross-validated Mean Accuracy: {cv_scores.mean():.2f} ± {cv_scores.std():.2f}")

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)


Accuracy: 1.00
Cross-validated Mean Accuracy: 1.00 ± 0.00

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19419
           1       1.00      1.00      1.00     25724

    accuracy                           1.00     45143
   macro avg       1.00      1.00      1.00     45143
weighted avg       1.00      1.00      1.00     45143


Confusion Matrix:
[[19419     0]
 [    7 25717]]
